In [1]:
from lego_robot import *
from slam_b_library import\
     filter_step, concatenate_transform
from project_landmarks import write_cylinders
from estimate_transform_correct_pose import\
     estimate_transform, apply_transform, correct_pose
from find_wall_pairs import\
     get_subsampled_points, get_corresponding_points_on_wall

##### Iterative Closest Point (ICP)
    -  ICP is one of the most used methods for aligning n (n>1) dimensional point sets based on an initial estimation.
    
    Iterate the steps of transforming the points, selecting point pairs, and estimating the transform. Returns the final transformation.

In [2]:
def get_icp_transform(world_points, iterations):
    # Equate all the transforms to identitiy initially
    overall_trafo = (1.0, 1.0, 0.0, 0.0, 0.0)
   
    for i in xrange(0, iterations):
        # Apply the transformation to the world points
        world_points = [apply_transform(overall_trafo, p) for p in world_points]
        # Get the corresponding wall pairs
        left, right = get_corresponding_points_on_wall(world_points)
        # Estimate the new transformation
        trafo = estimate_transform(left, right, fix_scale = True)
    
        if(trafo):
            # Concatenate the new transformation to the overall transformation
            overall_trafo = concatenate_transform(trafo, overall_trafo)
    
    return overall_trafo

In [3]:
if __name__ == '__main__':
    # The constants we used for the filter_step.
    scanner_displacement = 30.0
    ticks_to_mm = 0.349
    robot_width = 150.0

    # The start pose we obtained miraculously.
    pose = (1850.0, 1897.0, 3.717551306747922)

    # Read the logfile which contains all scans.
    logfile = LegoLogfile()
    logfile.read("robot4_motors.txt")
    logfile.read("robot4_scan.txt")

    # Iterate over all positions.
    out_file = file("icp_wall_transform.txt", "w")
    for i in xrange(len(logfile.scan_data)):
        # Compute the new pose.
        pose = filter_step(pose, logfile.motor_ticks[i],
                           ticks_to_mm, robot_width,
                           scanner_displacement)

        # Subsample points.
        subsampled_points = get_subsampled_points(logfile.scan_data[i])
        world_points = [LegoLogfile.scanner_to_world(pose, c)
                        for c in subsampled_points]

        # Get the transformation.
        # You may play withe the number of iterations here to see
        # the effect on the trajectory!
        trafo = get_icp_transform(world_points, iterations = 3)

        # Correct the initial position using trafo.
        pose = correct_pose(pose, trafo)

        # Write to file.
        # The pose.
        out_file.write("F %f %f %f\n" % pose)
        # Write the scanner points and corresponding points.
        write_cylinders(out_file, "W C",
            [apply_transform(trafo, p) for p in world_points])

    out_file.close()